In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#📎 You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm
from PIL import Image
import os
from pathlib import Path
import random
import matplotlib.pyplot as plt
import itertools

# Set up device and initialize random seed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Set random seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Path to the Alzheimer's dataset
DATA_ROOT = Path("/kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset")

In [ ]:
# Define class mapping for Alzheimer's dataset
class_map = {
    "No Impairment": 0,
    "Very Mild Impairment": 1,
    "Mild Impairment": 2,
    "Moderate Impairment": 3
}

image_paths = []
labels = []

# Loop over train and test directories
for folder in ["train", "test"]:
    folder_path = DATA_ROOT / folder
    for class_name, class_idx in class_map.items():
        class_dir = folder_path / class_name
        if class_dir.is_dir():
            for img_path in class_dir.glob("*"):
                if img_path.suffix.lower() in [".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff"]:
                    image_paths.append(str(img_path))
                    labels.append(class_idx)

print("Total images found:", len(image_paths))

# Create a DataFrame for the dataset
data_df = pd.DataFrame({
    "path": image_paths,
    "label": labels
})

data_df.head()

In [ ]:
# Image size for Alzheimer's classification (MRI images might require different image size)
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_EPOCHS = 20
LR = 1e-4
NUM_CLASSES = 4
N_SPLITS = 5  # 5-fold cross-validation

# Data augmentations (adjusted for MRI data)
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
class AlzheimerDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, "path"]
        label = self.df.loc[idx, "label"]

        img = Image.open(img_path)
        if img.mode != "RGB":
            img = img.convert("RGB")

        if self.transform:
            img = self.transform(img)

        return img, label

In [ ]:
def create_model(num_classes=NUM_CLASSES):
    # Pretrained Xception model from timm
    model = timm.create_model(
        "xception",
        pretrained=True,
        num_classes=num_classes
    )
    return model

In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss()

def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    running_correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * images.size(0)
        running_correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = running_correct / total
    return epoch_loss, epoch_acc


def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * images.size(0)
            running_correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = running_correct / total
    return epoch_loss, epoch_acc

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

fold_histories = []
fold_best_accs = []
fold_best_paths = []

print(f"Starting {N_SPLITS}-fold cross-validation...")

for fold, (train_idx, val_idx) in enumerate(
    skf.split(data_df["path"], data_df["label"]), 1
):
    print(f"\n========== Fold {fold}/{N_SPLITS} ==========")

    train_df = data_df.iloc[train_idx].reset_index(drop=True)
    val_df = data_df.iloc[val_idx].reset_index(drop=True)

    print("Train size:", len(train_df))
    print("Val size:", len(val_df))

    train_dataset = AlzheimerDataset(train_df, transform=train_transform)
    val_dataset = AlzheimerDataset(val_df, transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    model = create_model()
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="max", factor=0.5, patience=3, verbose=True
    )

    history = {
        "train_loss": [],
        "val_loss": [],
        "train_acc": [],
        "val_acc": []
    }

    best_val_acc = 0.0
    best_model_path = f"best_xception_alzheimer_fold{fold}.pth"

    for epoch in range(1, NUM_EPOCHS + 1):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate(model, val_loader, criterion, device)

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        scheduler.step(val_acc)

        print(
            f"Fold {fold} | Epoch [{epoch}/{NUM_EPOCHS}] "
            f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
            f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}"
        )

        # Save best model (highest validation accuracy for this fold)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"  -> New best model for fold {fold} saved with val_acc = {best_val_acc:.4f}")

    fold_histories.append(history)
    fold_best_accs.append(best_val_acc)
    fold_best_paths.append(best_model_path)

print("\nCross-validation complete.")
for i, acc in enumerate(fold_best_accs, 1):
    print(f"Fold {i} best val_acc: {acc:.4f}")
print(f"Mean val_acc over {N_SPLITS} folds: {np.mean(fold_best_accs):.4f}")

In [ ]:
# pick the fold with the highest validation accuracy
best_fold_index = int(np.argmax(fold_best_accs))  # 0-based index
best_fold = best_fold_index + 1
best_history = fold_histories[best_fold_index]

print(f"Best fold is Fold {best_fold} with val_acc = {fold_best_accs[best_fold_index]:.4f}")

epochs = range(1, NUM_EPOCHS + 1)

plt.figure(figsize=(14, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs, best_history["train_acc"], label=f"Fold {best_fold} Train Acc")
plt.plot(epochs, best_history["val_acc"], label=f"Fold {best_fold} Val Acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title(f"Train vs Val Accuracy (Fold {best_fold})")
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, best_history["train_loss"], label=f"Fold {best_fold} Train Loss")
plt.plot(epochs, best_history["val_loss"], label=f"Fold {best_fold} Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title(f"Train vs Val Loss (Fold {best_fold})")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Evaluate model using confusion matrix and classification report
all_labels = []
all_preds = []

for fold, (train_idx, val_idx) in enumerate(skf.split(data_df["path"], data_df["label"]), 1):
    val_df = data_df.iloc[val_idx].reset_index(drop=True)
    val_dataset = AlzheimerDataset(val_df, transform=val_transform)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    best_model_path = f"best_xception_alzheimer_fold{fold}.pth"

    model = create_model()
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    model.to(device)
    model.eval()

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

cm = confusion_matrix(all_labels, all_preds)
print("\nConfusion matrix:\n", cm)

print("\nClassification report:\n")
print(classification_report(all_labels, all_preds, target_names=list(class_map.keys())))

# Plot confusion matrix
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(6, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.tight_layout()

# Plot confusion matrix
plot_confusion_matrix(cm, classes=list(class_map.keys()), normalize=False, title="Confusion Matrix (Counts)")
plot_confusion_matrix(cm, classes=list(class_map.keys()), normalize=True, title="Confusion Matrix (Normalized)")

In [ ]:
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

class GradCAM:
    """
    Grad-CAM implementation for Xception model.
    Target layer: 'block12.rep.4' (last convolutional layer before fully connected layers).
    """
    def __init__(self, model, target_layer_name="block12.rep.4"):
        self.model = model
        self.model.eval()
        self.target_layer = dict([*self.model.named_modules()])[target_layer_name]

        self.gradients = None
        self.activations = None

        # forward hook: save activations
        self.forward_hook = self.target_layer.register_forward_hook(self._forward_hook)
        # backward hook: save gradients
        self.backward_hook = self.target_layer.register_backward_hook(self._backward_hook)

    def _forward_hook(self, module, input, output):
        # output shape: [N, C, H, W]
        self.activations = output

    def _backward_hook(self, module, grad_input, grad_output):
        # grad_output[0] shape: [N, C, H, W]
        self.gradients = grad_output[0]

    def generate(self, input_tensor, class_idx=None):
        """
        input_tensor: (1, 3, H, W)
        class_idx: target class index; if None, use predicted class
        """
        self.model.zero_grad()
        output = self.model(input_tensor)

        if class_idx is None:
            class_idx = torch.argmax(output, dim=1).item()

        loss = output[0, class_idx]
        loss.backward()

        # activations & gradients with batch dimension
        # shapes: [N, C, H, W]; we use only the first sample (N=1)
        activations = self.activations[0].detach()   # [C, H, W]
        gradients = self.gradients[0].detach()       # [C, H, W]

        # global average pooling on gradients -> [C, 1, 1]
        weights = torch.mean(gradients, dim=(1, 2), keepdim=True)
        # weighted sum of activations -> [H, W]
        cam = torch.sum(weights * activations, dim=0)

        cam = F.relu(cam)
        cam -= cam.min()
        cam /= (cam.max() + 1e-8)

        cam_np = cam.cpu().numpy()  # shape: [H, W]
        return cam_np

    def remove_hooks(self):
        self.forward_hook.remove()
        self.backward_hook.remove()


def show_gradcam_example(model, dataset, idx=0, target_layer="block12.rep.4"):
    """
    Show Grad-CAM heatmap for one example from a dataset.
    """
    # inverse normalization to display the image
    inv_normalize = transforms.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.225]
    )

    img_tensor, label = dataset[idx]
    input_tensor = img_tensor.unsqueeze(0).to(device)

    # create Grad-CAM object
    gradcam = GradCAM(model, target_layer_name=target_layer)
    cam = gradcam.generate(input_tensor)
    gradcam.remove_hooks()

    # prediction
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
        pred_idx = torch.argmax(output, dim=1).item()

    # prepare original image for plotting
    img_for_display = inv_normalize(img_tensor).clamp(0, 1).permute(1, 2, 0).cpu().numpy()

    # resize CAM to image size
    cam_uint8 = (cam * 255).astype(np.uint8)          # shape [H, W]
    cam_img = Image.fromarray(cam_uint8)              # grayscale image
    cam_resized = cam_img.resize(
        (img_for_display.shape[1], img_for_display.shape[0]),
        resample=Image.BILINEAR
    )
    cam_resized = np.array(cam_resized) / 255.0       # back to [0, 1]

    # Plot original, CAM, and overlay
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.imshow(img_for_display)
    plt.axis("off")
    plt.title(f"Original\nTrue: {target_names[label]}")

    plt.subplot(1, 3, 2)
    plt.imshow(cam_resized, cmap="jet")
    plt.axis("off")
    plt.title(f"Grad-CAM\nPred: {target_names[pred_idx]}")

    plt.subplot(1, 3, 3)
    plt.imshow(img_for_display)
    plt.imshow(cam_resized, cmap="jet", alpha=0.4)
    plt.axis("off")
    plt.title("Overlay")

    plt.tight_layout()
    plt.show()

In [ ]:
# Define target names for the Alzheimer's classes
target_names = ["No Impairment", "Very Mild Impairment", "Mild Impairment", "Moderate Impairment"]

# Rebuild the best fold's validation dataset
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
best_fold_index = int(np.argmax(fold_best_accs))  # 0-based index
best_fold = best_fold_index + 1

current_fold = 1
val_dataset_best_fold = None

# Retrieve validation set for the best fold
for train_idx, val_idx in skf.split(data_df["path"], data_df["label"]):
    if current_fold == best_fold:
        val_df_best = data_df.iloc[val_idx].reset_index(drop=True)
        val_dataset_best_fold = AlzheimerDataset(val_df_best, transform=val_transform)
        break
    current_fold += 1

# Load the best model from the best fold
best_model_path = f"best_xception_alzheimer_fold{best_fold}.pth"
best_model = create_model()
best_model.load_state_dict(torch.load(best_model_path, map_location=device))
best_model.to(device)
best_model.eval()

print(f"Showing Grad-CAM examples from Fold {best_fold}")

# Pick at least one index for each class ["No Impairment", "Very Mild Impairment", "Mild Impairment", "Moderate Impairment"]
num_classes = len(class_map)

# Find the first occurrence index of each class in the validation dataset
class_to_idx = {}  # label_int -> idx in dataset

for idx in range(len(val_dataset_best_fold)):
    _, label = val_dataset_best_fold[idx]
    label_int = int(label)
    if label_int not in class_to_idx:
        class_to_idx[label_int] = idx
    if len(class_to_idx) == num_classes:
        break

print("Chosen indices per class (label -> index):", class_to_idx)

# Run Grad-CAM on one example per class (if present in this fold)
for label_int, idx in sorted(class_to_idx.items()):
    class_name = list(class_map.keys())[label_int]
    print(f"\nGrad-CAM for class '{class_name}' (label={label_int}) at validation index {idx} (Fold {best_fold})")
    show_gradcam_example(best_model, val_dataset_best_fold, idx=idx, target_layer="block12.rep.4")